In [1]:
import os
os.chdir('/home/nmduy/CA683/Credit Project')
from fancyimpute import IterativeImputer as MICE
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [3]:
# data_dummy = pd.read_csv('filtered_application_train.csv')
# data_dummy = data_dummy.drop(['SK_ID_CURR', 'TARGET'], axis=1)

# categorical_count = data_dummy.select_dtypes('object').apply(pd.Series.nunique, axis = 0)
# categorical_name = list(categorical_count.keys())
# continuous_name = [x for x in list(data_dummy.columns) if x not in categorical_name]

In [4]:
data = pd.read_csv('encoded_train_rmflg_group_dropna.csv')
data_rm = data.drop(['SK_ID_CURR', 'TARGET'], axis=1) # remove target

In [6]:
##### PSEUDO MISSING #####
import random 
random.seed(1509)

data_pseudo = data_rm.copy()
pseudo_portion = 0.2
numb_pseudo = int(pseudo_portion * len(data_rm))
index = [i for i in range(len(data_rm))]

for col in list(data_rm.columns):
    pseudo_na_index = random.sample(index, numb_pseudo)
    data_pseudo[col][pseudo_na_index] = np.nan


In [8]:
imputed_data_pseudo = MICE().fit_transform(data_pseudo)
imputed_pd = pd.DataFrame(data=imputed_data_pseudo)
imputed_pd.columns = list(data_pseudo.columns)


In [31]:
imputed_pd.to_csv('initial_imputed_train.csv', index=False)

In [10]:
imputed_pd.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade,ORGANIZATION_TYPE_Transport,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA
0,1.0000,1.000000,0.00000,202500.000000,3.978864e+05,22531.937119,3.510000e+05,0.018801,12111.154414,637.0,...,-0.026294,-0.002336,0.0,0.128781,0.005435,0.000000,0.0,0.000000,0.0,0.0
1,0.0000,0.514795,0.00000,67500.000000,1.350000e+05,6750.000000,1.084180e+05,0.010032,12963.118663,225.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
2,1.0000,1.000000,0.00000,99000.000000,6.183956e+05,27517.500000,5.552343e+05,0.035792,14983.849330,1588.0,...,0.028492,0.001823,0.0,0.000000,0.000000,0.000000,0.0,0.032098,0.0,0.0
3,1.0000,0.000000,1.00000,243019.785837,1.560726e+06,41301.000000,1.404208e+06,0.035792,13778.000000,3130.0,...,0.000000,0.000000,0.0,0.000000,0.056916,0.024549,0.0,0.000000,0.0,0.0
4,0.8722,0.556790,-0.00529,360000.000000,1.530000e+06,42075.000000,1.530000e+06,0.003122,18850.000000,449.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.029676,0.0,0.096332,0.0,0.0


In [28]:
pred = []
lbl = []
for col in list(data_rm.columns):
    pseudo_na_index = np.asarray(pd.isnull(data_pseudo[col])).nonzero()[0]
    lbl += list(data_rm[col][pseudo_na_index])
    pred += list(imputed_pd[col][pseudo_na_index])

pred_np = np.asarray(pred)
lbl_np = np.asarray(lbl)
error = (np.square(pred_np - lbl_np)).mean()
print(f"Mean Square Error: {error}")

Mean Square Error: 416279890.1954319


In [32]:
a = imputed_pd.copy()

In [47]:
a['NAME_CONTRACT_TYPE'][a['NAME_CONTRACT_TYPE'] < 0.5] = 0
a['NAME_CONTRACT_TYPE'][a['NAME_CONTRACT_TYPE'] != 0] = 1
a['CODE_GENDER'][a['CODE_GENDER'] < 0.5] = 0
a['CODE_GENDER'][a['CODE_GENDER'] != 0] = 1
a['CNT_CHILDREN'][a['CNT_CHILDREN'] < 0] = 0
a['CNT_FAM_MEMBERS'][a['CNT_FAM_MEMBERS'] < 0] = 0
for col in list(a.columns):
    if 'NAME' in col or 'OCCUPATION' in col or 'WEEKDAY' in col or 'ORGANIZATION' in col:
        a[col][a[col] < 0.5] = 0
        a[col][a[col] != 0] = 1
    else:
        a[col][a[col] < 0] = 0

    


In [48]:
a.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade,ORGANIZATION_TYPE_Transport,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA
0,1.0,1.0,0.0,202500.000000,3.978864e+05,22531.937119,3.510000e+05,0.018801,12111.154414,637.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,67500.000000,1.350000e+05,6750.000000,1.084180e+05,0.010032,12963.118663,225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,99000.000000,6.183956e+05,27517.500000,5.552343e+05,0.035792,14983.849330,1588.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,1.0,243019.785837,1.560726e+06,41301.000000,1.404208e+06,0.035792,13778.000000,3130.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,360000.000000,1.530000e+06,42075.000000,1.530000e+06,0.003122,18850.000000,449.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
pred = []
lbl = []
for col in list(data_rm.columns):
    pseudo_na_index = np.asarray(pd.isnull(data_pseudo[col])).nonzero()[0]
    lbl += list(data_rm[col][pseudo_na_index])
    pred += list(a[col][pseudo_na_index])

pred_np = np.asarray(pred)
lbl_np = np.asarray(lbl)
error = (np.square(pred_np - lbl_np)).mean()
print(f"Mean Square Error: {error}")

Mean Square Error: 414630834.66379166


In [51]:
pred = {}
lbl = {}
for col in list(data_rm.columns):
    pseudo_na_index = np.asarray(pd.isnull(data_pseudo[col])).nonzero()[0]
    lbl[col] = list(data_rm[col][pseudo_na_index])
    pred[col] = list(a[col][pseudo_na_index])

In [61]:
def calculate_r_square(pred, lbl):
    # pred and lbl is numpy vector
    correlation_matrix = np.corrcoef(pred, lbl)
    correlation_xy = correlation_matrix[0,1]
    r_squared = correlation_xy**2
    return r_squared

def calculate_adjusted_r_square(pred, lbl, numb_ft):
    r_squared = calculate_r_square(pred, lbl)
    numb_obs = len(pred)
    result = 1 - (numb_obs-1)/(numb_obs-numb_ft-1)*(1-r_squared**2)
    return result


In [67]:
categorical_name = ['NAME_CONTRACT_TYPE', 'CODE_GENDER']
for col in list(a.columns):
    if 'NAME' in col or 'OCCUPATION' in col or 'WEEKDAY' in col or 'ORGANIZATION' in col:
        categorical_name += [col]

In [68]:
numb_ft = len(list(a.columns)) - 1
rs = {}
ars = {}
for col in list(lbl.keys()):
    if col not in categorical_name:
        pred_np = np.asarray(pred[col])
        lbl_np = np.asarray(lbl[col])
        rs[col] = calculate_r_square(pred_np, lbl_np)
        ars[col] = calculate_adjusted_r_square(pred_np, lbl_np, numb_ft)

In [69]:
rs

{'CNT_CHILDREN': 0.7394223801085207,
 'AMT_INCOME_TOTAL': 0.10149770992433313,
 'AMT_CREDIT': 0.8596905213605305,
 'AMT_ANNUITY': 0.5927066381913073,
 'AMT_GOODS_PRICE': 0.8637102279779647,
 'REGION_POPULATION_RELATIVE': 0.23932091409740996,
 'DAYS_BIRTH': 0.31820419955377655,
 'DAYS_EMPLOYED': 0.07217389903701019,
 'DAYS_REGISTRATION': 0.10595877189987835,
 'DAYS_ID_PUBLISH': 0.034556292368791194,
 'OWN_CAR_AGE': 0.01251066481071903,
 'CNT_FAM_MEMBERS': 0.81163763082965,
 'REGION_RATING_CLIENT': 0.7708559296942382,
 'REGION_RATING_CLIENT_W_CITY': 0.7594352324475622,
 'HOUR_APPR_PROCESS_START': 0.0883157442855389,
 'REG_REGION_NOT_LIVE_REGION': 0.559590626491772,
 'REG_REGION_NOT_WORK_REGION': 0.7247794717581054,
 'LIVE_REGION_NOT_WORK_REGION': 0.7259762684156277,
 'REG_CITY_NOT_LIVE_CITY': 0.5187940051616261,
 'REG_CITY_NOT_WORK_CITY': 0.6969203614434925,
 'LIVE_CITY_NOT_WORK_CITY': 0.6900091764630807,
 'EXT_SOURCE_2': 0.15361433528832824,
 'EXT_SOURCE_3': 0.06712878019119754,
 'OBS_3

In [70]:
acc = {}
for col in list(lbl.keys()):
    if col in categorical_name:
        pred_np = np.asarray(pred[col])
        lbl_np = np.asarray(lbl[col])
        dif = pred_np - lbl_np
        correct = np.where(dif == 0)[0]
        correct = len(correct)
        acc[col] = correct / len(dif)

In [71]:
acc

{'NAME_CONTRACT_TYPE': 0.9043166898746767,
 'CODE_GENDER': 0.7609906504873682,
 'NAME_TYPE_SUITE_Children': 0.9933359856773424,
 'NAME_TYPE_SUITE_Family': 0.9637457728267357,
 'NAME_TYPE_SUITE_Group of people': 0.9991048338969565,
 'NAME_TYPE_SUITE_Other_A': 0.9964442013129103,
 'NAME_TYPE_SUITE_Other_B': 0.9949522578078377,
 'NAME_TYPE_SUITE_Spouse, partner': 0.9823204694648896,
 'NAME_TYPE_SUITE_Unaccompanied': 0.9498458325044759,
 'NAME_INCOME_TYPE_Businessman': 0.9999502685498309,
 'NAME_INCOME_TYPE_Commercial associate': 0.9321414362442809,
 'NAME_INCOME_TYPE_Maternity leave': 1.0,
 'NAME_INCOME_TYPE_Pensioner': 0.9999254028247464,
 'NAME_INCOME_TYPE_State servant': 0.969067037994828,
 'NAME_INCOME_TYPE_Student': 1.0,
 'NAME_INCOME_TYPE_Unemployed': 1.0,
 'NAME_INCOME_TYPE_Working': 0.924731450169087,
 'NAME_EDUCATION_TYPE_Academic degree': 0.9992042967972946,
 'NAME_EDUCATION_TYPE_Higher education': 0.9364680724089914,
 'NAME_EDUCATION_TYPE_Incomplete higher': 0.9851054306743585,